In [1]:
import tensorflow as tf
import keras
from keras import models, layers

VGG16 autoencoder for anomaly detection

1. Convolution layers numbers = [2,2,3,3,3]
2. filters list = [64,128,256,512,512]
3. decoder filter list = [ ]

In [3]:
class ConvUnit(tf.keras.layers.Layer):
    def __init__(self, Filter, units, stride=1, activation='relu'):
        super().__init__()
        self.layers = [ 
            layers.Conv2D(filters=Filter, strides=stride, activation=activation, kernel_size=3, padding='same', use_bias=False) for _ in range(units)
        ]
        self.layers.append(layers.MaxPool2D(pool_size=(2, 2), strides=2))
    
    def call(self, inputs):
        z = inputs
        for layer in self.layers:
            z = layer(z)
        return z

class TConvUnit(tf.keras.layers.Layer):
    def __init__(self, Filter, units, stride=1, activation='relu'):
        super().__init__()
        self.layers=[
            layers.Conv2DTranspose(filters=Filter, strides=stride, activation=activation, kernel_size=3, padding='same', use_bias=False) for _ in range(units)
        ]
        self.layers.append(layers.UpSampling2D(size=(2, 2)))

    def call(self, inputs):
        z = inputs
        for layer in self.layers:
            z = layer(z)
        return z

# Define the encoder model
encoder = models.Sequential()
encoder.add(tf.keras.Input(shape=(224, 224, 3), name="Input Layer"))
prev_filter = 64
for unit in [2, 2, 3, 3, 3]:
    f = prev_filter if prev_filter != 512 else 512
    encoder.add(ConvUnit(Filter=f, units=unit))
    prev_filter = min(prev_filter * 2, 512)
encoder.summary()

# Define the decoder model
decoder = models.Sequential()
decoder.add(tf.keras.Input(shape=(7, 7, 512), name="Input layer decoder"))
prev_filter = 512
for unit in [3, 3, 2, 1]:
    f = prev_filter
    decoder.add(TConvUnit(Filter=int(f), units=unit))
    prev_filter = prev_filter // 2
decoder.add(layers.UpSampling2D(size=(2, 2)))
decoder.add(layers.Conv2D(filters=3, kernel_size=3, activation='sigmoid', padding='same'))
decoder.summary()

# Define the autoencoder
autoencoder = models.Sequential([encoder, decoder])
autoencoder.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
autoencoder.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_unit_5 (ConvUnit)          │ (None, 112, 112, 64)   │        38,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_unit_6 (ConvUnit)          │ (None, 56, 56, 128)    │       221,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_unit_7 (ConvUnit)          │ (None, 28, 28, 256)    │     1,474,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_unit_8 (ConvUnit)          │ (None, 14, 14, 512)    │     5,898,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_unit_9 (ConvUnit)          │ (None, 7, 7, 512)      │     7,077,888 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,710,464 (56.12 MB)

 Trainable params: 14,710,464 (56.12 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ ?                      │    14,710,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ ?                      │     9,955,011 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,665,475 (94.09 MB)

 Trainable params: 24,665,475 (94.09 MB)

 Non-trainable params: 0 (0.00 B)